# Imports

In [1]:
import joblib

In [2]:
# Chargement des modèles depuis le dossier 'models'
model_RF_ADC = joblib.load("../models/model_RF_ADC.joblib")
model_RF_support = joblib.load("../models/model_RF_support.joblib")
model_RF_jungler = joblib.load("../models/model_RF_jungler.joblib")
model_RF_top = joblib.load("../models/model_RF_top.joblib")
model_RF_mid = joblib.load("../models/model_RF_mid.joblib")
df_adc = joblib.load("../models/df_adc.joblib")
df_jungler = joblib.load("../models/df_jungler.joblib")
df_mid = joblib.load("../models/df_mid.joblib")
df_support = joblib.load("../models/df_support.joblib")
df_top = joblib.load("../models/df_top.joblib")
features_Bot = joblib.load("../models/features_Bot.joblib")
features_jungler = joblib.load("../models/features_jungler.joblib")
features_mid = joblib.load("../models/features_mid.joblib")
features_support = joblib.load("../models/features_support.joblib")
features_top = joblib.load("../models/features_top.joblib")
